In [ ]:
#import the required libraries
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont
import matplotlib.pyplot as plt
%matplotlib inline

#define the function
def show_custom_labels(model,bucket,photo, min_confidence):
     
    #call rekognition using boto3
    client=boto3.client('rekognition')

    #for inference upload your required image in s3 bucket
    #load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #calls DetectCustomLabels 
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},MinConfidence=min_confidence,ProjectVersionArn=model)
   
    #gives height and width of resultant image
    imgWidth, imgHeight = image.size
    print(imgWidth)
    print(imgHeight)
    
    #plotting of image, if you want big display increase arguments of figsize
    fig,ax = plt.subplots(1, 1, figsize=[8,8])
    
    draw = ImageDraw.Draw(image)
       
    # calculate and display bounding boxes for each detected custom label       
    print('Detected custom labels for ' + photo)    
    for customLabel in response['CustomLabels']:
        #prints the label name and confidence score
        print('Label ' + str(customLabel['Name'])) 
        print('Confidence ' + str(customLabel['Confidence']))
        
        if 'Geometry' in customLabel:
            #draws bounding boxes around objects
            box = customLabel['Geometry']['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']
              
            #selects the position of label on box
            draw.text((left,top), customLabel['Name'])
            
            #prints the geometery of box for json format
            print('Left: ' + '{0:.0f}'.format(left))
            print('Top: ' + '{0:.0f}'.format(top))
            print('Label Width: ' + "{0:.0f}".format(width))
            print('Label Height: ' + "{0:.0f}".format(height))
            
            
            points = ((left,top),(left + width, top),(left + width, top + height),(left , top + height),(left, top))
            
            #if label is a paper, draws a box with width of 2 and color corresponding to color code '#00d400'
            #if need thicker box, increase width, all labels have different color of boxes according to their color codes
            if str(customLabel['Name']) == 'paper':
                draw.line(points, fill='#00d400', width=2)
            if str(customLabel['Name']) == 'cardboard':
                draw.line(points, fill='#bc4100', width=2)
            if str(customLabel['Name']) == 'tissue':
                draw.line(points, fill='#fffe1c', width=2)
            if str(customLabel['Name']) == 'can':
                draw.line(points, fill='#221600', width=2)
            if str(customLabel['Name']) == 'bottle':
                draw.line(points, fill='#4b5a00', width=2)
            if str(customLabel['Name']) == 'plastic_bag':
                draw.line(points, fill='#ff32a7', width=2)
            if str(customLabel['Name']) == 'wrapper':
                draw.line(points, fill='#0004bf', width=2)
            if str(customLabel['Name']) == 'trash':
                draw.line(points, fill='#ffddfd', width=2)
            if str(customLabel['Name']) == 'wood':
                draw.line(points, fill='#030030', width=2)
            if str(customLabel['Name']) == 'compost':
                draw.line(points, fill='#cd4f39', width=2)
            
            #to show image
            ax.imshow(image)

    return len(response['CustomLabels'])

def main():
    
    #give the bucket which has photo along with image name. I have img46.jpg in folder'images'
    bucket="images-for-detection"
    photo="images/img46.jpg"
    
    
    #model arn with 10 classes
    #in rekognition, it's written under performance matrix
    model='arn:aws:rekognition:us-east-1:758820547797:project/416_416/version/416_416.2020-08-11T17.22.10/1597180930722'
    
    #set a high confidence level to get good results
    min_confidence=95
    
    label_count=show_custom_labels(model,bucket,photo, min_confidence)
    print("Custom labels detected: " + str(label_count))
    plt.show()
    

if __name__ == "__main__":
    main()
    
    